In [ ]:
import isaacgym

In [ ]:
from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed

In [ ]:
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, get_rlgames_env_creator
from rl_games.common import env_configurations, vecenv
from rl_games.torch_runner import Runner
from rl_games.algos_torch import model_builder

In [ ]:
import isaacgymenvs

In [ ]:
from omegaconf import DictConfig, OmegaConf

In [ ]:
cfg = OmegaConf.load("cfg/config.yaml")

In [ ]:
cfg.task_name="TrifingerNYU"
cfg.num_envs=1
cfg.task=OmegaConf.load("cfg/task/TrifingerNYU.yaml")

In [ ]:
def create_env_thunk(**kwargs):
        envs = isaacgymenvs.make(
            
            cfg.seed,
            cfg.task,
            cfg.num_envs,
            cfg.sim_device,
            cfg.rl_device,
            headless=True,
            cfg=cfg,
            **kwargs,
        )
        return envs

In [ ]:
# register the rl-games adapter to use inside the runner
vecenv.register('RLGPU',
                lambda config_name, num_actors, **kwargs: RLGPUEnv(config_name, num_actors, **kwargs))
env_configurations.register('rlgpu', {
    'vecenv_type': 'RLGPU',
    'env_creator': create_env_thunk,
})

In [ ]:
envs = create_env_thunk()

In [125]:
envs._fingertip_indices

[6, 13, 20]

In [186]:
fid = [5 ,12, 19]

In [196]:
jacobian_fingertip_linear = envs._jacobian[:, fid, :3, :]
jacobian_fingertip_linear = jacobian_fingertip_linear.view(
                envs.num_envs, 
                3 * envs._dims.NumFingers.value, 
                envs._dims.GeneralizedCoordinatesDim.value)

In [197]:
# get fingertip states
fingertip_state = envs._rigid_body_state[:, envs._fingertip_indices]
fingertip_velocity = fingertip_state[:, :, 7:10].reshape(envs.num_envs, 9)

q = envs._dof_position
dq = envs._dof_velocity

In [198]:
import trifinger_simulation
import numpy as np
import pinocchio as pin

In [199]:
platform = trifinger_simulation.TriFingerPlatform()
trifinger = platform.simfinger
rmodel = trifinger.kinematics.robot_model
rdata = trifinger.kinematics.data
ftip_id = trifinger.kinematics.tip_link_ids

In [200]:
q_np = q[0].cpu().numpy()
dq_np = dq[0].cpu().numpy()

In [201]:
pin.computeJointJacobians(rmodel, rdata, q_np)
pin.framesForwardKinematics(rmodel, rdata, q_np)
J = pin.getFrameJacobian(
    rmodel,
    rdata,
    ftip_id[1],
    pin.ReferenceFrame.LOCAL_WORLD_ALIGNED,
)[:3, :]

In [202]:
J @ dq_np

array([0.19653007, 0.06740448, 0.52209394])

In [203]:
print(jacobian_fingertip_linear[0] @ dq[0])

tensor([-0.1090,  0.1158,  0.3961,  0.1965,  0.0674,  0.5221, -0.0457, -0.1521,
         0.3961], device='cuda:0')


In [204]:
fingertip_velocity

tensor([[-0.0997,  0.1164, -0.0268,  0.1915,  0.0569, -0.0390, -0.0509, -0.1443,
         -0.0268]], device='cuda:0')